# Count connected tracks inside a cone

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
df_test = pd.read_csv('DS_2_test.csv', index_col=0)

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
zl = np.unique(df['Z'])
zmin = np.min(zl)
zmax = np.max(zl)
dz = np.diff(zl)[0]

In [202]:
print(df["X"].min(), df["X"].max())
print(df["Y"].min(), df["Y"].max())
print(df_test["X"].min(), df_test["X"].max())
print(df_test["Y"].min(), df_test["Y"].max())

27762.7578125 72240.3359375
3313.84375 76710.2421875
27762.7578125 72240.3359375
3313.84375 76710.2421875


In [204]:
np.arange(27700, 72400, 100)
np.arange(3300, 76900, 100)

array([27700, 27800, 27900, 28000, 28100, 28200, 28300, 28400, 28500,
       28600, 28700, 28800, 28900, 29000, 29100, 29200, 29300, 29400,
       29500, 29600, 29700, 29800, 29900, 30000, 30100, 30200, 30300,
       30400, 30500, 30600, 30700, 30800, 30900, 31000, 31100, 31200,
       31300, 31400, 31500, 31600, 31700, 31800, 31900, 32000, 32100,
       32200, 32300, 32400, 32500, 32600, 32700, 32800, 32900, 33000,
       33100, 33200, 33300, 33400, 33500, 33600, 33700, 33800, 33900,
       34000, 34100, 34200, 34300, 34400, 34500, 34600, 34700, 34800,
       34900, 35000, 35100, 35200, 35300, 35400, 35500, 35600, 35700,
       35800, 35900, 36000, 36100, 36200, 36300, 36400, 36500, 36600,
       36700, 36800, 36900, 37000, 37100, 37200, 37300, 37400, 37500,
       37600, 37700, 37800, 37900, 38000, 38100, 38200, 38300, 38400,
       38500, 38600, 38700, 38800, 38900, 39000, 39100, 39200, 39300,
       39400, 39500, 39600, 39700, 39800, 39900, 40000, 40100, 40200,
       40300, 40400,

In [90]:
df0 = df.query("brick_number==53")
df0['TZ'] = np.sqrt(1-df0.TX**2-df0.TY**2)
dx = df0.TX / df0.TZ * dz
dy = df0.TY / df0.TZ * dz
df0.loc[:, 'X1'] = df0['X'] + dx # self projected
df0.loc[:, 'Y1'] = df0['Y'] + dy

/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  
/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on

In [91]:
frm = df0.query("Z==47841")
print(frm.shape)

(1717, 16)


In [92]:
too = df0.query("Z==49134")
print(too.shape)

(1684, 16)


In [147]:

print(max(), np.cos(0.1))

0.999217599045 0.995004165278


In [195]:
cood = ["X", "Y", "Z"]; idz = 1/dz
def angd(evf, too):
    r0 = evf[cood].values
    x = too.query("(X-{}) ** 2 + (Y-{}) ** 2 < 2.5e8".format(r0[0], r0[1]))[cood].values - r0
    x /= np.linalg.norm(x, axis=1, keepdims=True)
    return np.sum(np.dot(x, evf[['TX','TY','TZ']].values) > np.cos(0.05))

In [196]:
def angl(efr, n):
    z0 = efr.iloc[0]['Z']
    if z0 >= zmax - (n-1)*dz:
        efr['cnt'] = 0
    else:
        eto = df.query("Z=={}".format(z0 + n*dz))
        efr['cnt'] = frm.apply(angd, axis=1, args=(eto,))
    return efr

In [197]:
angd(df0.query("Z==1293"), 1)

/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,ys,xs,YS,XS,TZ,X1,Y1,cnt
index,,,,,,,,,,,,,,,,,
1579,-999,30948.824219,75302.414062,1293.0,0.180050,-0.333445,2.945203,0.0,53,0.899940,0.968600,0.899940,0.968600,0.925417,31200.391861,74836.522053,NaN
5609,-999,37616.910156,35950.726562,1293.0,-0.351116,-0.055339,2.603913,0.0,53,0.996947,0.890248,0.996947,0.890248,0.934695,37131.197748,35874.174619,NaN
18473,-999,51582.601562,38366.109375,1293.0,-0.312202,-0.393899,1.811409,0.0,53,0.865684,0.911186,0.865684,0.911186,0.864508,51115.656337,37776.974973,NaN
29863,-999,66707.031250,6723.945312,1293.0,0.366332,0.053218,2.056914,0.0,53,0.997176,0.881679,0.997176,0.881679,0.928961,67216.920253,6798.018252,NaN
30873,-999,35349.039062,48902.328125,1293.0,0.154334,0.316797,2.512505,0.0,53,0.908793,0.976735,0.908793,0.976735,0.935853,35562.271249,49340.023335,NaN
33657,-999,56910.230469,45354.812500,1293.0,-0.327716,-0.222098,1.787803,0.0,53,0.952991,0.903018,0.952991,0.903018,0.918300,56448.794614,45042.090058,NaN
33717,-999,44031.695312,65015.957031,1293.0,0.302697,0.354092,2.753726,0.0,53,0.888588,0.916065,0.888588,0.916065,0.884869,44474.006516,65533.368450,NaN
52095,-999,58765.851562,30885.121094,1293.0,-0.090885,0.084710,2.735340,0.0,53,0.992875,0.991808,0.992875,0.991808,0.992252,58647.419109,30995.506155,NaN
52333,-999,57579.765625,6850.230469,1293.0,-0.031027,0.642643,1.769989,0.0,53,0.707719,0.999038,0.707719,0.999038,0.765537,57527.361065,7935.661447,NaN


In [185]:
%debug

> /opt/conda/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py(2198)norm()
   2196             # special case for speedup
   2197             s = (x.conj() * x).real
-> 2198             return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
   2199         else:
   2200             try:

ipdb> u
> <ipython-input-182-f0025cea3cd8>(5)angd()
      2 def angd(evf, too):
      3     r0 = evf[cood].values
      4     x = too.query("(X-{}) ** 2 + (Y-{}) ** 2 < 2.5e7".format(r0[0], r0[1]))[cood] - r0
----> 5     x /= np.linalg.norm(x, axis=1, keepdims=True)
      6     return np.sum(np.dot(x, evf[['TX','TY','TZ']].values) > np.cos(0.05))

ipdb> print(x)
               X        Y      Z
index                           
902      2089.17 -685.914 -46548
1253    -455.621 -2307.43 -46548
1351    -285.695  3151.38 -46548
2780    -1140.86  716.504 -46548
4896    -833.246 -1650.51 -46548
5582    -3776.62 -642.062 -46548
6361     2599.91  -10.207 -46548
15950   -3072.98  -1446.8 -46548
174

In [6]:
def base_n_away(n):
    rename = {"dfore":"dfore{}".format(n), "dback":"dback{}".format(n)}
    xdf = df.groupby("brick_number").apply(features, n)
    xdfo = xdf[['dfore', 'dback']].rename(columns=rename)
    xdfo.to_hdf("data-r{}.h5".format(n), "train")
    xdft = df_test.groupby("brick_number").apply(features, n)
    xdfot = xdft[['dfore', 'dback']].rename(columns=rename)
    xdfot.to_hdf("data-r{}.h5".format(n), "test")
    return n

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
with ProcessPoolExecutor(max_workers=4) as executor:
    rst = list(executor.map(base_n_away, (3,4,5,6)))